# Randomized Benchmarking: Simultaneous RB

In [ ]:
# Needs in terminal:
# $ quilc -S
# $ qvm -S

import numpy as np

from pyquil.api import get_benchmarker
from forest.benchmarking.randomized_benchmarking import *
from forest.benchmarking.stratified_experiment import *

%matplotlib inline

In [ ]:
from pyquil.api import get_qc, get_benchmarker
qc = get_qc("9q-square-noisy-qvm")
bm = get_benchmarker()

# Run RB experiments

## 1) Generate two RB experiments

In [ ]:
num_sequences = 10
depths = 3 * 2 ** np.arange(3, dtype=np.uint8)

expt = generate_rb_experiment(bm, (0, 5), depths, num_sequences)
expt2 = generate_rb_experiment(bm, [1], depths, num_sequences)
print(expt)

## 2) Run a single experiment on a QuantumComputer

In [ ]:
expt_results = acquire_rb_data(qc, expt, num_shots=100)

In [ ]:
expt_results[0]

## Run a simultaneous rb experiment on qubits 1 and 5

In [ ]:
simult_results = acquire_rb_data(qc, [expt, expt2], num_shots=100)

## 3) Fit the data

In [ ]:
from forest.benchmarking.analysis.fitting import make_figure

fit = fit_rb_results(simult_results[0])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
rb_decay = fit.params['decay']

In [ ]:
fit = fit_rb_results(simult_results[1])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
rb_decay = fit.params['decay'].value

## simultaneous unitarity

In [ ]:
num_sequences = 10
depths = 3 * 2 ** np.arange(3, dtype=np.uint8)

uexpt = generate_unitarity_experiment(bm, [5], depths, num_sequences, use_self_inv_seqs = True)
uexpt2 = generate_unitarity_experiment(bm, [1], depths, num_sequences)
simult_u_results = acquire_unitarity_data(qc, [uexpt, uexpt2], num_shots=100)

In [ ]:
from forest.benchmarking.analysis.fitting import make_figure

fit = fit_unitarity_results(simult_u_results[0])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['unitarity'].value

In [ ]:
fit = fit_unitarity_results(simult_u_results[1])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Shifted Purity")
unitarity = fit.params['unitarity'].value

A unitarity experiment with self-inverting sequences can be treated as an rb experiment

In [ ]:
populate_rb_survival_statistics(simult_u_results[0])
fit = fit_rb_results(simult_u_results[0])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
rb_decay = fit.params['decay'].value